# Pogo Calculator

## Introduction
PokemonGo is an AR mobile pokemon game that you can catch pokemon and use them for PlayerVsPlayer battles. Each pokemon has distinct base stats (Attack, Defense, Stamina) and with increasing level these stats scale. PokemonGo also incorporated individual values that gave each wild pokemon a combination of 3 numbers (one for each respective stat) from 0 to 15. As an example, a wild Pikachu could appear with an IV combination of 0 Attack IV, 15 Defense IV, 9 Stamina IV or 10 Attack IV, 7 Defense IV, 1 Stamina IV. 

Each pokemon can be leveled up to increase its stats and each level has its own multiple for scaling. The multiple for levels are termed CPM (Combat power) multipliers and using the formulas below you arrive at your pokemons total combat power and stat product from its base stats, individual values (4096 combinations), and CP multiplier(level). Combat power is relevant because it determines in what league a person can play that pokemon in. For example, 1500 being the ceiling for Great League, and 2500 being the ceiling for Ultra League.

## Goal

A buddy of mine really enjoys playing PokemonGo and he would often mention how tedious it was to regularly go to a website (that was often offline) to verify his caught pokemons rank amongst the variations and check how valuable it would be in a PvP scenario. The goal is to get a users pokemon and IV combination as input to compare against all other wild combinations at their highest potential and determine placement. My approach is to build a visualization for this idea and incorporate it into a Discord Bot that can be deployed onto a server. **I take no credit for the discovery of these formulas. A more in depth description and respective credits can be found on [gamepress.gg](https://gamepress.gg/pokemongo/pokemon-stats-advanced).**



#### Base values
Every species of Pokémons' base values are calculated from the base stats of the main pokemon games.

$$ Base Attack (PokemonGo) = 2 * round(\sqrt{Attack} * \sqrt{SpAttack} + \sqrt{Speed}) $$

$$ Base Defense (PokemonGo) = 2 * round(\sqrt{Defense} * \sqrt{SpDefense} + \sqrt{Speed}) $$

$$ Base Stamina (PokemonGo) = 2 * Hp $$


#### CPM
Combat Power multipliers are constant for each half level and although there is a formula to approxiamte listed below, I found a nifty excel file that came with the exact values listed. From here on, We will be using the words CP multiplier(CPM) and Level interchangeably. 

$$ CPM = (0.0095) \sqrt{Level} $$

#### Combat Power
Next we have the formula for finding the Combat Power of a specific pokemon and its individual values is 

$$ CP = \frac{(Base Attack + IV Attack) \sqrt{Base Defense + IV Defense} \sqrt{Base Stamina + IV Stamina} \ \ CPM^{2}}{10}$$

#### Stat Prodcut
Although CP is our capping variable, the most popular way of determining a pokemons potential damage and strength is using its stat product. We can use this approach to better determine each combinations placement.

$$ Stat Product = (Total Attack * CPM)(Total Defense * CPM)(Total Stamina * CPM) $$

In [160]:
!pip install --upgrade pip
!pip install -r requirements.txt

In [161]:
import pandas as pd
import numpy as np

# Using pandas to read three files.

pokemon_stats = pd.read_excel('excel/pokestats.xlsx')
iv_combinations = pd.read_excel('excel/ivPer.xlsx')
cp_multiplier = pd.read_excel('excel/Multiplier.xlsx')

# Venusaur, 15, 10, 7 will be our test case.

name = "Venusaur"
iv_attack = 15
iv_defense = 10
iv_stamina = 7

In [162]:
# Labeling columns and options for our data frames.

pokemon_stats.columns = ['Pokemon', 'Base Attack', 'Base Defense', 'Base Stamina']
pokemon_stats = pokemon_stats.set_index('Pokemon', drop=True)

iv_combinations.columns = ['Attack', 'Defense', 'Stamina']

cp_multiplier.columns = ['CPM', 'Level']

pd.set_option('display.float_format', lambda x: '%.3f' % x)

pokemon_stats
#iv_permutations.head()
#cp_multiplier.head()

,Base Attack,Base Defense,Base Stamina
Pokemon,,,
Bulbasaur,118,111,128
Ivysaur,151,143,155
Venusaur,198,189,190
Charmander,116,93,118
Charmeleon,158,126,151
...,...,...,...
Shaymin Land,210,210,225
Shaymin Sky,261,166,225
Arceus,238,238,236


In [163]:
# Getting our pokemons stats from the base values data frame.
att_Stat = pokemon_stats.loc[name, 'Base Attack']
def_Stat = pokemon_stats.loc[name, 'Base Defense']
sta_Stat = pokemon_stats.loc[name, 'Base Stamina']

# Slicing our base stats to combine with each permutation of wild individual values to get our total stats.
iv_combinations['Total Attack'] = att_Stat + iv_combinations['Attack']
iv_combinations['Total Defense'] = def_Stat + iv_combinations['Defense']
iv_combinations['Total Stamina'] = sta_Stat + iv_combinations['Stamina']

#att_Stat
#def_Stat
#sta_Stat
#iv_combinations

Now we can use the total attack, defense, stamina, and the pokemon league CP ceiling(1500) to reformat the Combat Power(CP) formula to solve for the nearest Level Multiplier(CPM or Combat Power Multiplier) that we need for each variation at their highest potential. 

$$ CPM = \sqrt{\frac{Ceiling(10)}{Total Attack * \sqrt{Total Defense} * \sqrt{Total Stamina}}}  $$



In [196]:
# Partitioning our variables to use in formula.
att_column = iv_combinations['Total Attack']
def_column_squared = iv_combinations['Total Defense'] ** (0.5)
sta_column_squared = iv_combinations['Total Stamina'] ** (0.5)

#Pokemon Go rounds CP down so we account for that by adding .9 to ceiling to get as close to 1501 as possible. 
iv_combinations['CPM'] = (15009.9 / (att_column * def_column_squared * sta_column_squared)) ** 0.5

# Using our CPM file we can find the nearest value for each Level.  
iv_combinations = pd.merge_asof(iv_combinations.sort_values(['CPM']), cp_multiplier, on='CPM')
iv_combinations = iv_combinations.rename(columns={'CPM': 'Approx. CPM'})

iv_combinations

,Attack,Defense,Stamina,Total Attack,Total Defense,Total Stamina,Approx. CPM,Level_x,Absolute CPM,Stat Product Attack,Stat Product Defense,Stat Product Stamina,Stat Product,CP,Approx. CPM,Level_y
0,15,15,15,213,204,205,0.587,19.000,0.582,124.025,118.785,119,1753149.039,1476.841,0.587,19.000
1,15,15,14,213,204,204,0.588,19.000,0.582,124.025,118.785,118,1738416.694,1473.234,0.588,19.000
2,15,14,15,213,203,205,0.588,19.000,0.582,124.025,118.203,119,1744555.171,1473.217,0.588,19.000
3,14,15,15,212,204,205,0.588,19.000,0.582,123.443,118.785,119,1744918.293,1469.907,0.588,19.000
4,15,14,14,213,203,204,0.588,19.000,0.582,124.025,118.203,118,1729895.044,1469.619,0.588,19.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,0,0,2,198,189,192,0.631,22.000,0.627,124.060,118.421,120,1762963.881,1480.750,0.631,22.000
4092,1,0,0,199,189,190,0.631,22.000,0.627,124.687,118.421,119,1757102.174,1480.458,0.631,22.000
4093,0,1,0,198,190,190,0.632,22.000,0.627,124.060,119.048,119,1757522.634,1476.910,0.632,22.000
4094,0,0,1,198,189,191,0.632,22.000,0.627,124.060,118.421,119,1748272.515,1476.889,0.632,22.000


In [166]:
# Finding the nearest CPM and naming it absolute.

level_column = iv_combinations['Level']
level_index_column = np.searchsorted(cp_multiplier['Level'], level_column)
cpm_column = cp_multiplier['CPM'].iloc[level_index_column]
cpm_column = cpm_column.reset_index(drop=True)
iv_combinations['Absolute CPM'] = cpm_column

iv_combinations.head()

,Attack,Defense,Stamina,Total Attack,Total Defense,Total Stamina,Approx. CPM,Level,Absolute CPM
0,15,15,15,213,204,205,0.587,19.000,0.582
1,15,15,14,213,204,204,0.588,19.000,0.582
2,15,14,15,213,203,205,0.588,19.000,0.582
3,14,15,15,212,204,205,0.588,19.000,0.582
4,15,15,13,213,204,203,0.588,19.000,0.582


#### Stat Product

In [177]:
iv_combinations['Stat Product Attack'] = iv_combinations['Total Attack'] * iv_combinations['Absolute CPM']
iv_combinations['Stat Product Defense'] = iv_combinations['Total Defense'] * iv_combinations['Absolute CPM']
iv_combinations['Stat Product Stamina'] = (iv_combinations['Total Stamina'] * iv_combinations['Absolute CPM']).astype(int)
iv_combinations['Stat Product'] = iv_combinations['Stat Product Attack'] * iv_combinations['Stat Product Defense'] * iv_combinations['Stat Product Stamina']

attack = iv_combinations['Total Attack']
defense = iv_combinations['Total Defense']
stamina = iv_combinations['Total Stamina']
CPM = iv_combinations['Absolute CPM']

iv_combinations['CP'] = (attack * (defense ** (1/2)) * (stamina **(1/2)) * (CPM ** 2)) / 10

iv_combinations


,Attack,Defense,Stamina,Total Attack,Total Defense,Total Stamina,Approx. CPM,Level,Absolute CPM,Stat Product Attack,Stat Product Defense,Stat Product Stamina,Stat Product,CP
0,0,14,11,198,203,201,0.613,21.000,0.612,121.207,124.268,123,1852645.839,1498.778
1,0,12,13,198,201,203,0.613,21.000,0.612,121.207,123.044,124,1849306.936,1498.778
2,0,15,10,198,204,200,0.613,21.000,0.612,121.207,124.880,122,1846635.814,1498.723
3,0,10,15,198,199,205,0.613,21.000,0.612,121.207,121.819,125,1845671.242,1498.631
4,3,15,14,201,204,204,0.605,20.500,0.605,121.570,123.384,123,1844968.210,1499.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,15,0,0,213,189,190,0.610,20.500,0.605,128.827,114.312,114,1678818.746,1476.539
4092,15,3,4,213,192,194,0.604,20.000,0.597,127.246,114.701,115,1678452.708,1467.106
4093,13,0,1,211,189,191,0.612,20.500,0.605,127.618,114.312,115,1677643.393,1466.519
4094,13,1,0,211,190,190,0.612,20.500,0.605,127.618,114.916,114,1671854.423,1466.539


In [176]:
# These are the placements for all combinations sorted by Stat Product.

iv_combinations= iv_combinations.sort_values('Stat Product', ascending=False)
iv_combinations = iv_combinations.reset_index(drop=True)

iv_combinations

,Attack,Defense,Stamina,Total Attack,Total Defense,Total Stamina,Approx. CPM,Level,Absolute CPM,Stat Product Attack,Stat Product Defense,Stat Product Stamina,Stat Product,CP
0,0,14,11,198,203,201,0.613,21.000,0.612,121.207,124.268,123,1852645.839,1498.778
1,0,12,13,198,201,203,0.613,21.000,0.612,121.207,123.044,124,1849306.936,1498.778
2,0,15,10,198,204,200,0.613,21.000,0.612,121.207,124.880,122,1846635.814,1498.723
3,0,10,15,198,199,205,0.613,21.000,0.612,121.207,121.819,125,1845671.242,1498.631
4,3,15,14,201,204,204,0.605,20.500,0.605,121.570,123.384,123,1844968.210,1499.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,15,0,0,213,189,190,0.610,20.500,0.605,128.827,114.312,114,1678818.746,1476.539
4092,15,3,4,213,192,194,0.604,20.000,0.597,127.246,114.701,115,1678452.708,1467.106
4093,13,0,1,211,189,191,0.612,20.500,0.605,127.618,114.312,115,1677643.393,1466.519
4094,13,1,0,211,190,190,0.612,20.500,0.605,127.618,114.916,114,1671854.423,1466.539


In [195]:
# Finding the users choice. In our exmaple it was Venusaur 15 / 10 / 7.

# Important to notice that index starts at 0. So Index + 1 is your pokemons rank. 

user_choice = iv_combinations[(iv_combinations['Attack'] == iv_attack) & (iv_combinations['Defense'] == iv_defense) & (iv_combinations['Stamina'] == iv_stamina)]
user_choice


,Attack,Defense,Stamina,Total Attack,Total Defense,Total Stamina,Approx. CPM,Level,Absolute CPM,Stat Product Attack,Stat Product Defense,Stat Product Stamina,Stat Product,CP
3853,15,10,7,213,199,197,0.597,19.500,0.590,125.646,117.388,116,1710919.811,1467.500
